In [4]:
%load_ext blackcellmagic

In [3]:
import pysentiment
# https://github.com/hanzhichao2000/pysentiment/issues/10 install error
import pandas
from IPython.display import clear_output
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [6]:
df = pandas.read_excel("Analyst_reports1.xls", sheet_name="Sheet1")

In [7]:
df.head().append(df.tail())

Company                                           Headline  \
0         Cisco  Cisco Posts Another Record Quarter With Growth...   
1         Cisco  Cisco Reports Record Quarterly Revenue and Pro...   
2         Cisco  Cisco's focus on software and services prepare...   
3         Cisco  Acquisition of Duo Security Fits With Cisco's ...   
4         Cisco  Cisco Readies for Cloud Networking Era With So...   
650  VMware Inc  VMware Reports Solid 1Q Results; Increases Ful...   
651  VMware Inc  Expanded opportunities in the on-premises data...   
652  VMware Inc  VMware Reports Solid 4Q Results; Provides Posi...   
653  VMware Inc  VMware Reports Solid 3Q Results; Increases Ful...   
654  VMware Inc  VMware and Amazon Web Services Announce Strate...   

             Author                 Date  \
0         Mark Cash          11/14/2018    
1         Mark Cash          08/15/2018    
2         Mark Cash          07/17/2018    
3         Mark Cash          08/02/2018    
4         Mark Cash          07/17/2018    
650  Timothy Feeney  2017-06-01 00:00:00   
651  Timothy Feeney          07/19/2016    
652  Timothy Feeney          01/27/2017    
653  Timothy Feeney          10/26/2016    
654  Timothy Feeney          10/13/2016    

                                       First Paragraph  \
0    Cisco's first-quarter results modestly beat ou...   
1    Cisco's record-setting fourth quarter closed o...   
2    Cisco remains a networking equipment behemoth ...   
3    This acquisition broadens Cisco's security sui...   
4    Upon taking a renewed look at Cisco Systems, w...   
650  VMware reported fiscal first-quarter revenue a...   
651  VMware is the market leader in hypervisor-base...   
652  VMware’s fourth-quarter fiscal 2016 results su...   
653  VMware’s third-quarter fiscal 2016 results cam...   
654  VMware and Amazon Web Services on Oct. 13 anno...   

                                        Last Paragraph  
0    The company guided the second quarter to have ...  
1    The company's guidance of a 5%-7% year-over-ye...  
2    Capital deployment has been in the form of div...  
3    This acquisition comes on the heels of Cisco c...  
4    Cisco remains a networking equipment behemoth ...  
650  For the first quarter, VMware’s consolidated r...  
651  Overall, VMware possesses disciplined capital-...  
652  For the fourth quarter, VMware’s consolidated ...  
653  For the quarter, VMWare’s consolidated revenue...  
654  We believe the partnership will enable VMware ...

In [9]:
df["Combined"] = df["Headline"] + df["First Paragraph"] + df["Last Paragraph"]

In [11]:
df.shape
# - 样本并不多

(655, 7)

In [12]:
df["Headline"].head(10)

0    Cisco Posts Another Record Quarter With Growth...
1    Cisco Reports Record Quarterly Revenue and Pro...
2    Cisco's focus on software and services prepare...
3    Acquisition of Duo Security Fits With Cisco's ...
4    Cisco Readies for Cloud Networking Era With So...
5    Cisco Continues to Shift Toward Recurring Subs...
6    Cisco Announces Solid Results and Aggressive S...
7    Cisco continues to transition to a software-de...
8                           Cisco Systems Under Review
9    Cisco's 1Q Results Are in Line; Top Line Shoul...
Name: Headline, dtype: object

In [ ]:
def get_Finance_score(row, LM):

    return pandas.Series(LM.get_score(LM.tokenize(text=row)))["Polarity"]

$$Polarity= (Pos-Neg)/(Pos+Neg)$$

In [18]:
lm = pysentiment.LM()
# https://github.com/hanzhichao2000/pysentiment/issues/11 static file error
lm.get_score(lm.tokenize(text=df["Headline"][0])).values()

dict_values([0, 0, 0.0, 0.0])

In [19]:
def get_General_score(row):
    sid=SIA()
    score=sid.polarity_scores(row)
    
    return score["compound"]

In [22]:
# $ git clone https://github.com/nltk/nltk_data.git
# 比较难下载，用 gitee
# 也很慢
import nltk
nltk.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10054] 远程主机强迫关闭了一个现有的连接。>


False

In [20]:
df["Headline"][0:100].apply(get_General_score)

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\lijiaxiang/nltk_data'
    - 'D:\\install\\miniconda\\nltk_data'
    - 'D:\\install\\miniconda\\share\\nltk_data'
    - 'D:\\install\\miniconda\\lib\\nltk_data'
    - 'C:\\Users\\lijiaxiang\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
nltk.download("vader_lexicon")

In [ ]:
pandas.DataFrame(df["Company"],df["Date"]).to_dict()

In [9]:
dataframe=pandas.DataFrame([df["Company"],df["Date"],df["Author"]])

LM=pysentiment.LM()


for COL in  ["Headline",'First Paragraph','Last Paragraph',"Combined"]:
    dataframe=dataframe.append(
        pandas.Series(df[COL].apply(get_Finance_score,args=([LM])),name="Finance "+COL)
    )
    
    dataframe=dataframe.append(
        pandas.Series(df[COL].apply(get_General_score),name="General "+COL)
    )
    
dataframe.T

Company                 Date               Author  \
0                      Cisco          11/14/2018             Mark Cash   
1                      Cisco          08/15/2018             Mark Cash   
2                      Cisco          07/17/2018             Mark Cash   
3                      Cisco          08/02/2018             Mark Cash   
4                      Cisco          07/17/2018             Mark Cash   
5                      Cisco          05/16/2018         Brian Colello   
6                      Cisco          02/14/2018         Brian Colello   
7                      Cisco          02/14/2018         Brian Colello   
8                      Cisco          02/12/2018         Brian Colello   
9                      Cisco          11/15/2017       Ilya Kundozerov   
10                     Cisco          06/09/2017       Ilya Kundozerov   
11                     Cisco          11/02/2017          Michael Wong   
12                     Cisco          10/23/2017       Ilya Kundozerov   
13                     Cisco          09/27/2017          Michael Wong   
14                     Cisco          08/21/2017       Ilya Kundozerov   
15                     Cisco          08/16/2017       Ilya Kundozerov   
16                     Cisco  2017-06-28 00:00:00      Ilya Kundozerov   
17                     Cisco          06/09/2017       Ilya Kundozerov   
18                     Cisco          05/18/2017       Ilya Kundozerov   
19                     Cisco          05/02/2017       Ilya Kundozerov   
20                     Cisco          04/26/2017          Michael Wong   
21                     Cisco          04/07/2017          Michael Wong   
22                     Cisco          02/15/2017       Ilya Kundozerov   
23                     Cisco          12/07/2016       Ilya Kundozerov   
24                     Cisco          01/25/2017       Ilya Kundozerov   
25                     Cisco          11/16/2016       Ilya Kundozerov   
26                     Cisco          08/18/2016       Ilya Kundozerov   
27                    Oracle          11/16/2018          John Barrett   
28                    Oracle          11/16/2018          John Barrett   
29                    Oracle          09/17/2018   William Fitzsimmons   
..                       ...                  ...                  ...   
625  O'Reilly Automotive Inc          11/01/2016           Zain Akbari   
626  O'Reilly Automotive Inc  2016-10-26 00:00:00          Zain Akbari   
627  O'Reilly Automotive Inc          08/02/2016           Zain Akbari   
628               VMware Inc          11/29/2018             Mark Cash   
629               VMware Inc  2018-08-24 00:00:00            Mark Cash   
630               VMware Inc          08/03/2018             Mark Cash   
631               VMware Inc          08/03/2018             Mark Cash   
632               VMware Inc          07/02/2018         Brian Colello   
633               VMware Inc          06/01/2018           Neil Macker   
634               VMware Inc  2018-03-01 00:00:00          Neil Macker   
635               VMware Inc          03/01/2018           Neil Macker   
636               VMware Inc          02/12/2018           Neil Macker   
637               VMware Inc          01/29/2018       Ilya Kundozerov   
638               VMware Inc  2018-01-29 00:00:00      Ilya Kundozerov   
639               VMware Inc  2018-01-26 00:00:00      Ilya Kundozerov   
640               VMware Inc          01/04/2018       Ilya Kundozerov   
641               VMware Inc  2017-12-13 00:00:00      Ilya Kundozerov   
642               VMware Inc  2017-11-30 00:00:00      Ilya Kundozerov   
643               VMware Inc          01/10/2018       Ilya Kundozerov   
644               VMware Inc          11/02/2017       Ilya Kundozerov   
645               VMware Inc          08/24/2017       Ilya Kundozerov   
646               VMware Inc          08/15/2017       Ilya Kundozerov   
647               VMware 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(pandas.DataFrame([dataframe.T["Finance Last Paragraph"],dataframe.T["General Last Paragraph"]]).T)

In [ ]:
dataframe

In [10]:
dataframe.T.to_csv("Sentiments_Final_really.csv",sep=",",index=False)

In [ ]:
{k.strip() for k in sorted(dataframe.T["Author"].unique())}

In [ ]:
x=dataframe.T["Date"]

In [ ]:
def clean(row):
    if type(row)==datetime.datetime:
        return row
    row=datetime.datetime.strptime(row.strip(),"%M/%d/%Y")
    return row

In [ ]:
x.apply(clean)

In [ ]:
import datetime

In [ ]:
x.values

In [ ]:
df["Date"]=df["Date"].apply(clean)

In [ ]:
df[df["Company"]=="Kroger"]

In [ ]:
dataframe.T[dataframe.T["Company"]=="Apple"].T[352].T

In [11]:
import random

In [15]:
random.randint(1,575)

541